In [98]:
import json
import pandas as pd

In [99]:
# function to get shortened wiki articles

N_SENTENCES=3
def get_sentences(x):
    n = N_SENTENCES + 1
    sentences = x.split('. ')
    try:
        return '. '.join(sentences[:n])
    except:
        return '. '.join(sentences)
def count_sentences(x):
    try:
        return len(x.split('. '))
    except:
        return -1

In [100]:
# read pickles

wiki_en = pd.read_pickle('data/wiki_en_summaries.pkl')
wiki_fr = pd.read_pickle('data/wiki_fr_summaries.pkl')
bnf_table = pd.read_pickle('data/bnf_table_summaries.pkl')
bnf_tags = pd.read_pickle('data/bnf_tags.pkl')

named_subjects = pd.read_pickle('data/named_subjects.pkl')

# this line will change 
age_gender = pd.read_json('data/age_gender_labeles.json',orient='records')

In [101]:
# get name-id df (exploded)

named_subjects = named_subjects[['id','title','names']].explode('names')
named_subjects = named_subjects.set_index('id')
named_subjects = named_subjects.rename(columns = {'names':'name'})

# get name-id df (nested in list)

id_img = named_subjects.drop('title', axis=1)
id_img = id_img.reset_index()
id_img = pd.DataFrame(id_img.groupby('name')['id'].apply(list))
id_img = id_img.rename(columns={'id':'id_list'})

In [102]:
# organize dataframes, shorten texts, rename columns

wiki_en['wiki_en_text'] = wiki_en.summary.apply(get_sentences)
wiki_fr['wiki_fr_text'] = wiki_fr.summary_fr.apply(get_sentences)

wiki_en = wiki_en[['name','wiki_en_text','url','weight']]
wiki_fr = wiki_fr[['name','wiki_fr_text','url_fr','weight_fr']]
bnf_summaries = bnf_table[['name','bnf_link','gender','note']]


wiki_en = wiki_en.rename(columns={'url':'wiki_en_link', 'weight':'weight_en'})
wiki_fr = wiki_fr.rename(columns={'url':'wiki_fr_link'})

In [103]:
# link name and id defined in age_gender df

named_age_gender = named_subjects.merge(age_gender[['age','gender','id']], on='id').\
rename(columns={'age':'age_estimate', 'gender':'gender_estimate'})


# if age_gender file is re-writtened, 
#id_name = pd.read_pickle('data/id_name.pkl').rename(columns={'id':'id_list'})
#named_age_gender = id_name.merge(age_gender[['age','gender','id']], on='name').rename(columns={'age':'age_estimate', 'gender':'gender_estimate'})


In [104]:
# merge everything

data_final = pd.merge(bnf_summaries,wiki_en,how='outer')
data_final = pd.merge(data_final,wiki_fr,how='outer')
data_final = pd.merge(data_final, named_age_gender, how='outer')
data_final = data_final.merge(id_img, on='name',how='outer')
data_final = data_final.merge(bnf_tags, on='name',how='outer')


In [105]:
# redefine rank <<<<<< THIS WILL CHANGE 


data_final.weight_en.fillna(-1, inplace=True)
data_final.weight_fr.fillna(-1, inplace=True)

data_final['weight'] = data_final[['weight_en','weight_fr']].mean(axis=1)

In [106]:
data_final = data_final.sort_values(by='weight', ascending=False)

In [107]:
data_final = data_final[['id','name','gender','id_list', 'weight','note', 'bnf_link', 'wiki_en_text', 'wiki_en_link',
        'wiki_fr_text', 'url_fr', 'age_estimate','gender_estimate','tags']]

In [108]:
data_final = data_final.drop(data_final.loc[data_final.id.isna() & data_final.id_list.isna()].index)

In [109]:
data_final.loc[data_final.id.isna(), 'id'] = data_final.loc[data_final.id.isna(), 'id_list'].map(lambda x: x[0])

In [110]:
import re

In [111]:
indexes = data_final.name.map(lambda x: re.search('(-17[\d]{2}|-18[1234]{1}[\d]{1})', x)).dropna().index

In [112]:
data_final.name.map(lambda x: re.search('(-17[\d]{2}|-18[1234]{1}[\d]{1})', x)).dropna()

1573    <re.Match object; span=(32, 37), match='-1794'>
1362    <re.Match object; span=(23, 28), match='-1833'>
1870    <re.Match object; span=(33, 38), match='-1810'>
1685    <re.Match object; span=(14, 19), match='-1842'>
1876    <re.Match object; span=(13, 18), match='-1842'>
1337    <re.Match object; span=(18, 23), match='-1821'>
973     <re.Match object; span=(25, 30), match='-1813'>
958     <re.Match object; span=(15, 20), match='-1814'>
Name: name, dtype: object

In [113]:
indexes

Int64Index([1573, 1362, 1870, 1685, 1876, 1337, 973, 958], dtype='int64')

In [114]:
data_final[data_final.index.isin(indexes)]

,id,name,gender,id_list,weight,note,bnf_link,wiki_en_text,wiki_en_link,wiki_fr_text,url_fr,age_estimate,gender_estimate,tags
1573,https://gallica.bnf.fr/ark:/12148/btv1b53187673w,"Robespierre, Maximilien de (1758-1794)",masculin,[https://gallica.bnf.fr/ark:/12148/btv1b531876...,69.0,Avocat. - Homme politique. - Révolutionnaire,http://data.bnf.fr/ark:/12148/cb11922216p#about,Maximilien François Marie Isidore de Robespier...,https://en.wikipedia.org/wiki/Maximilien_Robes...,"Maximilien de Robespierre, ou Maximilien Robes...",https://fr.wikipedia.org/wiki/Maximilien_de_Ro...,38.010203,0.479522,"[avocat, hommepolitique, France]"
1362,https://gallica.bnf.fr/ark:/12148/btv1b53118972r,"Niépce, Nicéphore (1765-1833)",masculin,[https://gallica.bnf.fr/ark:/12148/btv1b531189...,14.0,Inventeur de la photographie,http://data.bnf.fr/ark:/12148/cb12068198f#about,Joseph Nicéphore Niépce (French: [nisefɔʁ njɛp...,https://en.wikipedia.org/wiki/Nic%C3%A9phore_N...,"Joseph « Nicéphore » Niépce, né le 7 mars 1765...",https://fr.wikipedia.org/wiki/Nic%C3%A9phore_N...,43.498299,0.501600,"[inventeur, France]"
1870,https://gallica.bnf.fr/ark:/12148/btv1b530664193,"Éon, Charles de Beaumont d' (1728-1810)",masculin,[https://gallica.bnf.fr/ark:/12148/btv1b530664...,6.5,"Diplomate, célèbre par le mystère qu'il laissa...",http://data.bnf.fr/ark:/12148/cb119865877#about,NaN,NaN,"Charles d'Éon de Beaumont, dit le chevalier d'...",https://fr.wikipedia.org/wiki/Charles_d%27%C3%...,26.078685,0.358721,"[diplomate, France]"
1685,https://gallica.bnf.fr/ark:/12148/btv1b530506058,Stendhal (1783-1842),masculin,[https://gallica.bnf.fr/ark:/12148/btv1b530506...,4.5,"Écrivain. - Certains ouvrages sont signés ""Hen...",http://data.bnf.fr/ark:/12148/cb119255047#about,Marie-Henri Beyle (French: [bɛl]; 23 January 1...,https://en.wikipedia.org/wiki/Stendhal,NaN,NaN,48.573742,0.082885,"[écrivain, France]"
1876,https://gallica.bnf.fr/ark:/12148/btv1b531542687,Raymond (18..-1842),NaN,[https://gallica.bnf.fr/ark:/12148/btv1b531542...,2.5,NaN,NaN,St. Raymond's Church is a parish church under ...,https://en.wikipedia.org/wiki/St._Raymond%27s_...,NaN,NaN,43.035933,0.047171,NaN
1337,https://gallica.bnf.fr/ark:/12148/btv1b531574340,Napoléon Ier (1769-1821 ; empereur des Français),masculin,[https://gallica.bnf.fr/ark:/12148/btv1b531072...,-1.0,Empereur des Français (1804-1814 et 1815),http://data.bnf.fr/ark:/12148/cb12008245w#about,NaN,NaN,NaN,NaN,38.712262,0.163517,[France]
973,https://gallica.bnf.fr/ark:/12148/btv1b53071606p,"Junot, Jean Andoche (1771-1813 ; duc d'Abrantès)",masculin,[https://gallica.bnf.fr/ark:/12148/btv1b530716...,-1.0,Général,http://data.bnf.fr/ark:/12148/cb120014134#about,NaN,NaN,NaN,NaN,27.294958,0.167698,"[général, France]"
958,https://gallica.bnf.fr/ark:/12148/btv1b53098086r,Joséphine (1763-1814 ; impératrice des Français),féminin,[https://gallica.bnf.fr/ark:/12148/btv1b530980...,-1.0,Chanteuse de dance music,http://data.bnf.fr/ark:/12148/cb14562750s#about,NaN,NaN,NaN,NaN,28.043095,0.525836,[chanteuse]


In [115]:
data_final['name'] = data_final.name.map(lambda x: re.sub('\?', '', x))

In [116]:
data_final = data_final[~data_final.index.isin(indexes)]

In [117]:
data_final = data_final.drop(data_final[data_final.name == 'Nicolas II (1868-1918 ; empereur de Russie)'].index)

In [118]:
data_final.to_pickle('data/merged_dataframe.pkl')